# Modelowanie wartości odszkodowań

### Estymowanie metodą estymatora największej wiarogodności

In [3]:
import numpy as np
import pandas as pd
from scipy import stats

In [4]:
data = pd.read_csv("https://github.com/ndzadz/mgr/blob/main/data_us_1990_2022.csv?raw=true",encoding="latin1",sep=";",header=0)

In [5]:
p95 = np.percentile(data[data["Insured Damages, Adjusted (\'000 US$)"].isna()==0]["Insured Damages, Adjusted (\'000 US$)"],95)
p95

5786907.8999999985

In [6]:
X = data[(data["Insured Damages, Adjusted (\'000 US$)"].isna()==0)&(data["Insured Damages, Adjusted (\'000 US$)"]<p95)]['Insured Damages, Adjusted (\'000 US$)']

#### Rozkład wykładniczy

$\hat{\lambda}=\frac{n}{\sum\limits_{i=1}^n x_i}$

In [7]:
lambda_hat = len(X)/sum(X)
print(lambda_hat)

1.395177366818116e-06


In [8]:
exp_res = stats.kstest(X, stats.expon.cdf,args=(lambda_hat,))
exp_res.pvalue

0.0

#### Rozkład lognormalny

$\hat{\mu} = \frac{\sum\limits_{i=1}^n\ln x_i}{n}
\\
\hat{\sigma}^2 = \frac{\sum\limits_{i=1}^n\left(\ln x_i-\frac{\sum_{i=1}^n\ln x_i}{n}\right)^2}{n}$

In [9]:
mu_hat = sum(np.log(X))/len(X)
sigma_sq_hat = sum(np.square(np.log(X)-mu_hat))/len(X)
print(mu_hat,sigma_sq_hat)

12.621668063027776 2.3407817241437945


In [10]:
lognorm_res = stats.kstest(X, stats.lognorm.cdf,args=(mu_hat,sigma_sq_hat))
lognorm_res.pvalue

8.73454306055164e-294

#### Rozkład gamma

$\hat{\lambda}=\left(\frac{1}{n}\displaystyle{\sum_{i=1}^n}x_i^{\hat{k}}\right)^{\frac{1}{\hat{k}}}
\\
\hat{k}=\frac{n}{\frac{1}{\hat{\lambda}}\displaystyle{\sum_{i=1}^{n}}x_i^{\hat{k}}\ln x_i-\displaystyle{\sum_{i=1}^{n}}\ln x_i}$

In [11]:
from scipy.special import digamma, gamma

In [32]:
import numpy as np
from scipy.special import digamma, gamma, polygamma

def gamma_mle(data):
    """Funkcja oblicza estymator największej wiarygodności dla rozkładu gamma."""
    n = len(data)
    s = np.log(np.mean(data)) - np.mean(np.log(data))
    r = np.log(n) - np.sum(digamma(data)) / n
    x0 = np.array([s, r])

    # Definicja funkcji i pochodnej log-likelihood dla rozkładu gamma
    def loglik(theta):
        return n * (theta[1] * np.log(theta[0]) - gamma(theta[1]) + (theta[1] - 1) * np.mean(np.log(data))) + n * np.log(gamma(theta[1]))

    def dloglik(theta):
        d1 = n * (np.log(theta[0]) - digamma(theta[1]))
        d2 = n * (theta[1] / theta[0] - np.mean(np.log(data)) + np.log(theta[0]) - polygamma(0, theta[1]))
        return np.array([d1, d2])

    # Implementacja algorytmu Newtona-Raphsona
    tol = 1e-6
    max_iter = 100
    iter = 0
    diff = tol + 1
    while iter < max_iter and diff > tol:
        theta = x0 - np.linalg.inv(hessian(x0, dloglik)) @ dloglik(x0)
        diff = np.linalg.norm(theta - x0)
        x0 = theta
        iter += 1

    return x0[0], x0[1]

def hessian(x, f):
    """Funkcja oblicza hesjan dla funkcji f w punkcie x."""
    n = len(x)
    hess = np.zeros((n, n))
    eps = 1e-6
    for i in range(n):
        for j in range(i, n):
            e_i = np.zeros(n)
            e_j = np.zeros(n)
            e_i[i] = eps
            e_j[j] = eps
            hess[i, j] = (f(x + e_i + e_j) - f(x + e_i) - f(x + e_j) + f(x)) / (eps ** 2)
            hess[j, i] = hess[i, j]
    return hess


In [31]:
from scipy.optimize import fsolve
from scipy.special import psi

In [56]:
def gamma_mle_equations(params, X):
    alpha, beta = params
    n = len(X)
    eq1 = alpha/np.mean(X) - beta
    eq2 = np.log(np.mean(X)/np.power(np.prod(X),1/n))-np.log(alpha)+psi(alpha)
    return eq1, eq2

alpha, beta = fsolve(gamma_mle_equations, [1,1], args=(X))
print(alpha,beta)

1.0 1.0


C:\Users\ndzad\AppData\Local\Temp/ipykernel_12980/3568942317.py:5: RuntimeWarning: divide by zero encountered in log
  eq2 = np.log(np.mean(X)/np.power(np.prod(X),1/n))-np.log(alpha)+psi(alpha)


In [55]:
np.prod(X)

inf

#### Rozkład Weibulla

$\hat{\lambda}=\left(\frac{1}{n}\displaystyle{\sum_{i=1}^n}x_i^{\hat{k}}\right)^{\frac{1}{\hat{k}}}
\\
\hat{k}=\frac{n}{\frac{1}{\hat{\lambda}}\displaystyle{\sum_{i=1}^{n}}x_i^{\hat{k}}\ln x_i-\displaystyle{\sum_{i=1}^{n}}\ln x_i}$

In [44]:
def weibull_mle_equations(params, X):
    k, lamb = params
    n = len(X)
    eq1 = np.power(np.sum(np.power(X, k)) / n, 1/k) - lamb
    eq2 = n / (np.sum(np.power(X, k)*np.log(X)) / lamb - np.sum(np.log(X))) - k
    return [eq1, eq2]

k, lamb = fsolve(weibull_mle_equations, [1,1], args=(X))
print(k,lamb)

-0.07922893559021643 275203.25662999536
